Late time kilonova

In this code we will post-processes the results for NSM simulations perform by Winnet.

-Libraries

In [1]:
#libraries

#Data
import pandas as pd
#import h5py

#Plots
import matplotlib.pyplot as plt

#fits
#from scipy.optimize import curve_fit

#math
import numpy as np

#Constants
#from scipy.constants import speed_of_light as c
#from scipy.constants import Avogadro as N_A
#from scipy.constants import Boltzmann as kb
#from scipy.constants import Stefan_Boltzmann as Sb

#Usefull
from tqdm.notebook import tqdm
import os
from scipy.interpolate import interp1d
import pynucastro as pyna
#%matplotlib widget

-Reading data without HDF5 format

In [2]:


folder = 'Example_NSM_dyn_ejecta_rosswog_varios_t_sinalpha\snaps'
files = os.listdir(folder)

abundances_winnet=[]
time_Winnet=[]
temperature_winnet=[]
density_winnet=[]

for f in tqdm(files):

    abundances_winnet.append(np.loadtxt(os.path.join(folder, f), skiprows=3))
    time_Winnet.append(float(np.loadtxt(os.path.join(folder, f), skiprows=1,max_rows=1, usecols=0)))
    temperature_winnet.append(float(np.loadtxt(os.path.join(folder, f), skiprows=1,max_rows=1, usecols=1)))
    density_winnet.append(float(np.loadtxt(os.path.join(folder, f), skiprows=1,max_rows=1, usecols=2)))

abundances_winnet = np.array(abundances_winnet)
time_Winnet = np.array(time_Winnet) 

  0%|          | 0/554 [00:00<?, ?it/s]

-Interpolation

In [3]:
initial_time=1 #seconds
final_time=24*60*60 #seconds
N_steps=10000
time = np.exp(np.linspace(np.log(initial_time), np.log(final_time), N_steps))
abundances_time= np.empty((abundances_winnet.shape[1],N_steps))
for i in tqdm(range(abundances_winnet.shape[1])):
    spline_interp = interp1d(time_Winnet, abundances_winnet[:,i,3], kind='linear')
    abundances_time[i] = spline_interp(time)
spline_interp_temp=interp1d(time_Winnet,temperature_winnet,kind='linear')
spline_interp_density=interp1d(time_Winnet,density_winnet,kind='linear')
temperature=spline_interp_temp(time)
density=spline_interp_density(time)

  0%|          | 0/6757 [00:00<?, ?it/s]

-Funtion that finds the nuclei and return X_i(t)

In [4]:

def abundance_time_of_nuclei_n_z(n,z):
    
    idx_start = np.searchsorted(abundances_winnet[0,:,1], z, side='left')
    idx_end = np.searchsorted(abundances_winnet[0,:,1], z, side='right')
    if idx_end>idx_start:
        j=np.searchsorted(abundances_winnet[0,:,0][idx_start:idx_end],n)+idx_start
        if abundances_winnet[0,:,0][j]==n and abundances_winnet[0,:,1][j]==z:
            return abundances_time[j]
        else:
            return np.zeros(N_steps)
    else:
        return np.zeros(N_steps)

-Propeties of nucleis

In [35]:
nuclear_data=pd.read_csv('Nuclear_data\Mass\winvne_v2.0.dat',skiprows=lambda x: not (x>7854 and (x-7855)%4==0),delim_whitespace=True,names=['name','A','Z','N','spin','Mass excess (Mev)','source'])
def index_winv_v2(z,n):
        
    idx_start = np.searchsorted(nuclear_data['Z'], z, side='left')
    idx_end = np.searchsorted(nuclear_data['Z'], z, side='right')
    if idx_end>idx_start:
        j=np.searchsorted(nuclear_data['N'][idx_start:idx_end],n)+idx_start
        if nuclear_data['N'][j]==n and nuclear_data['Z'][j]==z:
            return j
        else:
            return 'None'
    else:
        return 'None'

-Rates

In [83]:
nuclear_rates_chapter_1=pd.read_csv('Nuclear_data\decays\Reaclib_18_9_20', skiprows=lambda x: not( x>2 and x<22447 and (x-3)%3==0),sep='\s+',header=None,names=['reactants','products','source','Q value (Mev)'])
nuclear_rates_chapter_2=pd.read_csv('Nuclear_data\decays\Reaclib_18_9_20', skiprows=lambda x: not( x>=22447 and x<87001 and (x-22446)%3==0),sep='\s\s+',header=None,names=['reactants','product1','product2','source','Q value (Mev)'])

C:\Users\Diego Hernandez\AppData\Local\Temp\ipykernel_15056\133996992.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  nuclear_rates_chapter_2=pd.read_csv('Nuclear_data\decays\Reaclib_18_9_20', skiprows=lambda x: not( x>=22447 and x<87001 and (x-22446)%3==0),sep='\s\s+',header=None,names=['reactants','product1','product2','source','Q value (Mev)'])


In [84]:
nuclear_rates_chapter_2

,reactants,product1,product2,source,Q value (Mev)
0,d,n,p,an06nv,-2.22457
1,d,n,p,an06nv,-2.22457
2,d,n,p,an06nv,-2.22457
3,t,n,d,nk06nv,-6.25737
4,t,n,d,nk06nv,-6.25737
...,...,...,...,...,...
21513,rg336,nrg335,pkrF v,-1.59131e+00,NaN
21514,rg337,nrg336,pkrF v,-3.14131e+00,NaN
21515,rg338,nrg337,pkrF v,-1.79131e+00,NaN
21516,cn285,he4ds281,wc12s,9.32510e+00,NaN
